In [1]:
import keras 
from keras.layers import Dense
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import to_categorical
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer
import io
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
np.random.seed(420) 

In [3]:
def create_feedforward_network(model, hidden_sizes, input_shape):
    count = 0
    for size in hidden_sizes:
        if count == 0:
            model.add(Dense(size, activation='relu', input_shape=input_shape))
            count+=1
        else:
            model.add(Dense(size, activation='relu'))

In [4]:
def create_convolutional_network(model, conv_sizes, fc_sizes, input_shape):
    #structure: conv, conv, pool, dropout, and so on
    count = 0
    for size in hidden_sizes:
        num_filters, kernel_size, stride = size[0]
        if count == 0:
            model.add(Convolution2D(num_filters, kernel_size=(kernel_size, kernel_size), strides = (stride, strides), padding='same', activation='relu', input_shape=input_shape))
            count+=1
        else:
            model.add(Convolution2D(num_filters, kernel_size=(kernel_size, kernel_size), strides = (stride, strides), padding='same', activation='relu'))
        num_filters, kernel_size, stride = size[1]
        model.add(Convolution2D(num_filters, kernel_size=(kernel_size, kernel_size), strides = (stride, strides), padding='same', activation='relu'))
        pool_size, stride = size[2]
        model.add(MaxPooling2D(pool_size=(pool_size, pool_size), strides=(stride, stride)))
        droput_prob = size[3]
        model.add(Dropout(dropout_prob))

    model.add(Flatten())    
    create_feedforward_network(model, fc_sizes)
    return model

In [5]:
def create_network(hidden_sizes, num_outputs, do_regression, use_fc=True, conv_sizes=None, input_shape=None, optimizer='adam'):
    model = Sequential()
    if use_fc:
        create_feedforward_network(model, hidden_sizes, input_shape)
    else:
        create_convolutional_network(model, conv_sizes, input_shape, hidden_sizes)
    if do_regression:
        model.add(Dense(num_outputs))
        model.compile(loss='mean_squared_error', optimizer=optimizer)
    else:
        model.add(Dense(num_outputs, activation='softmax'))
        model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])

    return model


In [7]:
def get_fc_data():
    ''' 
    assume data is of the form:
    each row of X is [x1, x2, a, b] where c = a+bi and x1, x2 are the mapped point 
    Y is a vector with c values that correspond to the rows of X
    '''
    X = np.load('X.npy')
    Y = np.load('Y.npy')
    Y = to_categorical(Y)
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
    return X_train, X_test, y_train, y_test


In [8]:
def get_conv_data():
    Y = np.load('c')#batch size by 2 matrix
    X = np.zeros((Y.shape[0], 512, 512, 1))#images
    base = 'data/julia'
    for i in range(Y.shape[0]):
        img = np.load(base+str(i)+'.npy')
        X[i] = img
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
    return X_train, X_test, y_train, y_test

In [9]:
def cross_val(model, X, y, num_folds, classification=False, fit_params=dict()):
    if classification:
        kfold = StratifiedKFold(n_splits=num_folds, random_state=None, shuffle=True)
        scorer = accuracy_score
    else:
        kfold = KFold(n_splits=num_folds, random_state=None, shuffle=True)
        scorer = mean_squared_error
    return cross_val_score(model, X, y, cv=kfold, scoring=make_scorer(scorer), fit_params=fit_params)

In [10]:
#for fully connected neural net training data set: each row of x is [x1, x2, a, b] where c = a+bi and x1, x2 are the mapped point 
Xtrain_fc, Xtest_fc, ytrain_fc, ytest_fc = get_fc_data()
fc_model = create_network([512, 256, 128], 2, False, input_shape=[4])

In [11]:
fc_model.fit(Xtrain_fc, ytrain_fc, batch_size=2048, shuffle=True, steps_per_epoch=None, epochs=10)

Epoch 1/10
4412030/4412030 [==============================] - 15s 3us/step - loss: 0.1038 - acc: 0.9532
Epoch 2/10
4412030/4412030 [==============================] - 14s 3us/step - loss: 0.0764 - acc: 0.9657
Epoch 3/10
4412030/4412030 [==============================] - 14s 3us/step - loss: 0.0692 - acc: 0.9689
Epoch 4/10
4412030/4412030 [==============================] - 14s 3us/step - loss: 0.0649 - acc: 0.9711
Epoch 5/10
4412030/4412030 [==============================] - 14s 3us/step - loss: 0.0624 - acc: 0.9720
Epoch 6/10
4412030/4412030 [==============================] - 14s 3us/step - loss: 0.0597 - acc: 0.9733
Epoch 7/10
4412030/4412030 [==============================] - 14s 3us/step - loss: 0.0578 - acc: 0.9741
Epoch 8/10
4412030/4412030 [==============================] - 14s 3us/step - loss: 0.0564 - acc: 0.9747
Epoch 9/10
4412030/4412030 [==============================] - 14s 3us/step - loss: 0.0548 - acc: 0.9754
Epoch 10/10
4412030/4412030 [==============================] - 1

In [12]:
fc_model.evaluate(Xtest_fc, ytest_fc)

2173090/2173090 [==============================] - 104s 48us/step


[0.052116959238146382, 0.97657345070843826]

In [ ]:
fit_params={'batch_size':2048, 'shuffle':True, steps_per_epoch=None, epochs=10}

In [ ]:
X = np.load('basic_data/X.npy')
Y = np.load('basic_data/Y.npy')
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
y_test.shape

In [ ]:
#TODOS:
'''
make auto encoder architectures, figure out math for predicting latents (VAE style?)


'''

In [14]:
with open('data/meta/cvalues.txt') as f:
    content = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
content = [x.strip() for x in content] 

In [ ]:
def autoencoder(down_conv_sizes, up_conv_sizes, optimizer='adam'):